# KoBert(데이터 5배수 증가)
> "NLP"

- toc:true
- branch: master
- badges: true
- comments: true
- author: 전북대학교 통계학과 이강철
- categories: [python]
- hide :false
- published: true

# 진행상황 

* 데이터 증강을 하지 않고 모델을 적합한 결과 `test auccary`가 `87%` 가량 밖에 나오지가 않음 따라서 데이터 증강 기법을 이용하여 데이터를 증강시킨 후 모델을 훈련시켰다.

* 데이터 증강을 각각 5 배수와 10배수를 하여 모델을 적합한 결과 10 배수는 너무 과적합이고 5배수가 가장 적당한 것 같다.

 # GPU 용량확인 

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
cd /content/drive/MyDrive/Colab Notebooks

/content/drive/MyDrive/Colab Notebooks


`런타임 초기화 하고 아래코드 돌려서 gpu용량 확인하고 코드돌리자....

In [3]:
import torch
import gc
gc.collect()

torch.cuda.empty_cache()

In [4]:
# memory footprint support libraries/code
!ln -sf /opt/bin/nvidia-smi /usr/bin/nvidia-smi
!pip install gputil
!pip install psutil
!pip install humanize

import psutil
import humanize
import os
import GPUtil as GPU

GPUs = GPU.getGPUs()
# XXX: only one GPU on Colab and isn’t guaranteed
gpu = GPUs[0]
def printm():
    process = psutil.Process(os.getpid())
    print("Gen RAM Free: " + humanize.naturalsize(psutil.virtual_memory().available), " |     Proc size: " + humanize.naturalsize(process.memory_info().rss))
    print("GPU RAM Free: {0:.0f}MB | Used: {1:.0f}MB | Util {2:3.0f}% | Total     {3:.0f}MB".format(gpu.memoryFree, gpu.memoryUsed, gpu.memoryUtil*100, gpu.memoryTotal))
printm()

  Created wheel for gputil: filename=GPUtil-1.4.0-py3-none-any.whl size=7411 sha256=265a58fab9769879aad95af44bc121053df9ba43ea3457390f08907ba63dc90c
  Stored in directory: /root/.cache/pip/wheels/6e/f8/83/534c52482d6da64622ddbf72cd93c35d2ef2881b78fd08ff0c
Successfully built gputil
Gen RAM Free: 25.7 GB  |     Proc size: 1.2 GB
GPU RAM Free: 16280MB | Used: 0MB | Util   0% | Total     16280MB


# Colab 환경설정

!pip install gluonnlp pandas tqdm   
!pip install mxnet
!pip install sentencepiece==0.1.91
!pip install transformers==4.8.2
!pip install torch

# github에서 KoBERT 파일을 로드 및 KoBERT모델 불러오기

In [5]:
!pip install 'git+https://github.com/SKTBrain/KoBERT.git#egg=kobert_tokenizer&subdirectory=kobert_hf'

  Cloning https://github.com/SKTBrain/KoBERT.git to /tmp/pip-install-zbmx6a28/kobert-tokenizer_1b2b0dbf3d81413aab84e885b2c156b0
  Running command git clone -q https://github.com/SKTBrain/KoBERT.git /tmp/pip-install-zbmx6a28/kobert-tokenizer_1b2b0dbf3d81413aab84e885b2c156b0
  Created wheel for kobert-tokenizer: filename=kobert_tokenizer-0.1-py3-none-any.whl size=4649 sha256=8a7cffaa4f8945bb0ada70ab31238fa1f42c00fc3b3eaa7848e4dcd72f840d49
  Stored in directory: /tmp/pip-ephem-wheel-cache-fwcu0bxa/wheels/10/b4/d9/cb627bbfaefa266657b0b4e8127f7bf96d27376fa1a23897b4
Successfully built kobert-tokenizer


In [6]:
!pip install git+https://git@github.com/SKTBrain/KoBERT.git@master

  Cloning https://****@github.com/SKTBrain/KoBERT.git (to revision master) to /tmp/pip-req-build-zgcsi_uh
  Running command git clone -q 'https://****@github.com/SKTBrain/KoBERT.git' /tmp/pip-req-build-zgcsi_uh
     |████████████████████████████████| 132 kB 3.2 MB/s 
     |████████████████████████████████| 344 kB 24.7 MB/s 
     |████████████████████████████████| 47.3 MB 1.4 MB/s 
     |████████████████████████████████| 4.5 MB 34.9 MB/s 
     |████████████████████████████████| 1.2 MB 41.2 MB/s 
     |████████████████████████████████| 3.8 MB 29.3 MB/s 
     |████████████████████████████████| 895 kB 48.6 MB/s 
     |████████████████████████████████| 596 kB 49.6 MB/s 
     |████████████████████████████████| 6.5 MB 35.3 MB/s 
     |████████████████████████████████| 67 kB 4.2 MB/s 
     |████████████████████████████████| 79 kB 5.6 MB/s 
     |████████████████████████████████| 8.6 MB 39.4 MB/s 
     |████████████████████████████████| 127 kB 46.1 MB/s 
  Created wheel for kobert: filename=kob

In [7]:
from kobert import get_pytorch_kobert_model
from kobert_tokenizer import KoBERTTokenizer
tokenizer = KoBERTTokenizer.from_pretrained('skt/kobert-base-v1')
bertmodel, vocab  = get_pytorch_kobert_model()

Downloading:   0%|          | 0.00/363k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/244 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/432 [00:00<?, ?B/s]

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'XLNetTokenizer'. 
The class this function is called from is 'KoBERTTokenizer'.


using cached model. /content/drive/MyDrive/Colab Notebooks/.cache/kobert_v1.zip
using cached model. /content/drive/MyDrive/Colab Notebooks/.cache/kobert_news_wiki_ko_cased-1087f8699e.spiece


#  필요한 라이브러리 불러오기

In [8]:
import torch
from torch import nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import gluonnlp as nlp
import numpy as np
from tqdm import tqdm, tqdm_notebook
import pandas as pd

#transformers
from transformers import AdamW
from transformers.optimization import get_cosine_schedule_with_warmup
from transformers import BertModel

#GPU 사용 시
device = torch.device("cuda:0")

# 데이터셋 불러오기

In [9]:
#collapse-hide
import pandas as pd
data = pd.read_csv('kobert입력데이터(n=5).csv')

In [10]:
#collapse-hide
data.head()

,document,topic,text
0,1,15,존경 지지 주택 전시관 입점 업체 임차인 주택 전시관 업체 심정 강제 철거 업체 계...
1,1,15,존경 전시관 입점 업체 임차인 전시관 업체 심정 호소 강제 업체 계약 관계 통보 피...
2,1,15,존경 감사 주택 전체 입점 업체 임차인 주택 전시관 남용 심정 소속 강제 임대료 감...
3,1,15,존경 주택 전시관 입점 업체 임차인 주택 전시관 업체 심정 호소 강제 철거 계약 관...
4,1,15,존경 지지 주택 전시관 입점 업체 임차인 주택 전시관 업체 심정 호소 강제 철거 업...


In [11]:
#collapse-hide
data["topic"] = data["topic"]-1

In [12]:
#collapse-hide
data.head()

,document,topic,text
0,1,14,존경 지지 주택 전시관 입점 업체 임차인 주택 전시관 업체 심정 강제 철거 업체 계...
1,1,14,존경 전시관 입점 업체 임차인 전시관 업체 심정 호소 강제 업체 계약 관계 통보 피...
2,1,14,존경 감사 주택 전체 입점 업체 임차인 주택 전시관 남용 심정 소속 강제 임대료 감...
3,1,14,존경 주택 전시관 입점 업체 임차인 주택 전시관 업체 심정 호소 강제 철거 계약 관...
4,1,14,존경 지지 주택 전시관 입점 업체 임차인 주택 전시관 업체 심정 호소 강제 철거 업...


In [13]:
#collapse-hide
data_list = []
for ques, label in zip(data['text'], data['topic'])  :
    data = []   
    data.append(ques)
    data.append(str(label))

    data_list.append(data)

#  입력 데이터셋을 토큰화하기

In [14]:
class BERTDataset(Dataset):
    def __init__(self, dataset, sent_idx, label_idx, bert_tokenizer,vocab, max_len,
                 pad, pair):
   
        transform = nlp.data.BERTSentenceTransform(
            bert_tokenizer, max_seq_length=max_len,vocab=vocab, pad=pad, pair=pair)
        
        self.sentences = [transform([i[sent_idx]]) for i in dataset]
        self.labels = [np.int32(i[label_idx]) for i in dataset]

    def __getitem__(self, i):
        return (self.sentences[i] + (self.labels[i], ))
         

    def __len__(self):
        return (len(self.labels))

## 파라미터 셋팅

In [15]:
max_len = 200
batch_size = 32
warmup_ratio = 0.1
num_epochs = 10
max_grad_norm = 1
log_interval = 200
learning_rate =  4e-5

# Train data & Test data





In [16]:
#train & test 데이터로 나누기
from sklearn.model_selection import train_test_split

dataset_train, dataset_test = train_test_split(data_list, test_size=0.2, shuffle=True, random_state=34)

In [17]:
tok=tokenizer.tokenize
data_train = BERTDataset(dataset_train, 0, 1, tok, vocab, max_len, True, False)
data_test = BERTDataset(dataset_test,0, 1, tok, vocab,  max_len, True, False)

## 토큰화 및 패딩, 정수 인코딩 진행

In [18]:
train_dataloader = torch.utils.data.DataLoader(data_train, batch_size=batch_size, num_workers=4)
test_dataloader = torch.utils.data.DataLoader(data_test, batch_size=batch_size, num_workers=4)

# KoBERT 모델 구현

In [19]:
class BERTClassifier(nn.Module):
    def __init__(self,
                 bert,
                 hidden_size = 768,
                 num_classes=23,   ##클래스 수 조정##
                 dr_rate=None,
                 params=None):
        super(BERTClassifier, self).__init__()
        self.bert = bert
        self.dr_rate = dr_rate
                 
        self.classifier = nn.Linear(hidden_size , num_classes)
        if dr_rate:
            self.dropout = nn.Dropout(p=dr_rate)
    
    def gen_attention_mask(self, token_ids, valid_length):
        attention_mask = torch.zeros_like(token_ids)
        for i, v in enumerate(valid_length):
            attention_mask[i][:v] = 1
        return attention_mask.float()

    def forward(self, token_ids, valid_length, segment_ids):
        attention_mask = self.gen_attention_mask(token_ids, valid_length)
        
        _, pooler = self.bert(input_ids = token_ids, token_type_ids = segment_ids.long(), attention_mask = attention_mask.float().to(token_ids.device),return_dict=False)
        if self.dr_rate:
            out = self.dropout(pooler)
        return self.classifier(out)

In [20]:
#BERT 모델 불러오기
model = BERTClassifier(bertmodel,  dr_rate=0.3).to(device)
 
#optimizer와 schedule 설정
no_decay = ['bias', 'LayerNorm.weight']
optimizer_grouped_parameters = [
    {'params': [p for n, p in model.named_parameters() if not any(nd in n for nd in no_decay)], 'weight_decay': 1e-6},
    {'params': [p for n, p in model.named_parameters() if any(nd in n for nd in no_decay)], 'weight_decay': 0.0}
]

optimizer = AdamW(optimizer_grouped_parameters, lr=learning_rate)
loss_fn = nn.CrossEntropyLoss() # 다중분류를 위한 대표적인 loss func

t_total = len(train_dataloader) * num_epochs
warmup_step = int(t_total * warmup_ratio)

scheduler = get_cosine_schedule_with_warmup(optimizer, num_warmup_steps=warmup_step, num_training_steps=t_total)

#정확도 측정을 위한 함수 정의
def calc_accuracy(X,Y):
    max_vals, max_indices = torch.max(X, 1)
    train_acc = (max_indices == Y).sum().data.cpu().numpy()/max_indices.size()[0]
    return train_acc
    
train_dataloader

/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:309: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,


In [21]:
#collapse-hide
train_history=[]
test_history=[]
loss_history=[]
for e in range(num_epochs):
    train_acc = 0.0
    test_acc = 0.0
    model.train()
    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(train_dataloader)):
        optimizer.zero_grad()
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length= valid_length
        label = label.long().to(device)
        out = model(token_ids, valid_length, segment_ids)
         
        #print(label.shape,out.shape)
        loss = loss_fn(out, label)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_grad_norm)
        optimizer.step()
        scheduler.step()  # Update learning rate schedule
        train_acc += calc_accuracy(out, label)
        if batch_id % log_interval == 0:
            print("epoch {} batch id {} loss {} train acc {}".format(e+1, batch_id+1, loss.data.cpu().numpy(), train_acc / (batch_id+1)))
            train_history.append(train_acc / (batch_id+1))
            loss_history.append(loss.data.cpu().numpy())
    print("epoch {} train acc {}".format(e+1, train_acc / (batch_id+1)))
    #train_history.append(train_acc / (batch_id+1))
    
    model.eval()
    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(test_dataloader)):
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length= valid_length
        label = label.long().to(device)
        out = model(token_ids, valid_length, segment_ids)
        test_acc += calc_accuracy(out, label)
    print("epoch {} test acc {}".format(e+1, test_acc / (batch_id+1)))
    test_history.append(test_acc / (batch_id+1))

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:8: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  


  0%|          | 0/2000 [00:00<?, ?it/s]

epoch 1 batch id 1 loss 3.2182071208953857 train acc 0.0
epoch 1 batch id 201 loss 3.0333094596862793 train acc 0.052705223880597014
epoch 1 batch id 401 loss 2.5394673347473145 train acc 0.11050498753117206
epoch 1 batch id 601 loss 2.003541946411133 train acc 0.2413685524126456
epoch 1 batch id 801 loss 1.3171145915985107 train acc 0.3445302746566791
epoch 1 batch id 1001 loss 1.361488938331604 train acc 0.41561563436563437
epoch 1 batch id 1201 loss 0.974885880947113 train acc 0.4704933388842631
epoch 1 batch id 1401 loss 0.9846394658088684 train acc 0.5114204139900071
epoch 1 batch id 1601 loss 0.3865899443626404 train acc 0.5452841973766396
epoch 1 batch id 1801 loss 0.7350060939788818 train acc 0.5720433092726264
epoch 1 train acc 0.59415625


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:31: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`


  0%|          | 0/500 [00:00<?, ?it/s]

...

epoch 9 test acc 0.9469474999999999


  0%|          | 0/2000 [00:00<?, ?it/s]

epoch 10 batch id 1 loss 0.0060175699181854725 train acc 1.0
epoch 10 batch id 201 loss 0.026890743523836136 train acc 0.9734141791044776
epoch 10 batch id 401 loss 0.06891006231307983 train acc 0.9737375311720698
epoch 10 batch id 601 loss 0.0034921150654554367 train acc 0.9752495840266223
epoch 10 batch id 801 loss 0.021551022306084633 train acc 0.9754213483146067
epoch 10 batch id 1001 loss 0.038401391357183456 train acc 0.9757117882117882
epoch 10 batch id 1201 loss 0.0011323075741529465 train acc 0.9758534554537885
epoch 10 batch id 1401 loss 0.002252324717119336 train acc 0.9758877587437544
epoch 10 batch id 1601 loss 0.02398933656513691 train acc 0.9757183010618363
epoch 10 batch id 1801 loss 0.18408672511577606 train acc 0.9756038312048861
epoch 10 train acc 0.9751875


  0%|          | 0/500 [00:00<?, ?it/s]

epoch 10 test acc 0.94668


In [22]:
#collapse-hide
import pandas as pd

In [25]:
#collapse-hide
test_history

[0.8024825,
 0.8954825,
 0.925875,
 0.939305,
 0.944385,
 0.9479299999999999,
 0.948465,
 0.9481975,
 0.9469474999999999,
 0.94668]

In [26]:
#collapse-hide
total_train = pd.DataFrame()
total_train["train_history"] = train_history
total_train["loss_history"] = loss_history

total_test = pd.DataFrame()

total_test["test_history"] = test_history

total_train.to_csv("데이터증강(n=5) train.csv")

total_test.to_csv("데이터증강(n=5) test.csv")

# 학습된 모델을 이용한 test data set 예측 

## 예측 함수 구현

In [48]:
def predict(predict_sentence):

    data = [predict_sentence, '0']
    dataset_another = [data]

    another_test = BERTDataset(dataset_another, 0, 1, tok, vocab, max_len, True, False)
    test_dataloader = torch.utils.data.DataLoader(another_test, batch_size=batch_size, num_workers=4)
    
    model.eval()

    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(test_dataloader):
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)

        valid_length= valid_length
        label = label.long().to(device)

        out = model(token_ids, valid_length, segment_ids)


        test_eval=[]
        for i in out:
            logits=i
            logits = logits.detach().cpu().numpy()
            test_eval.append(np.argmax(logits))
        return test_eval

## loop를 통해 test data set의 전체 문서의 적용 

In [83]:
length = len(dataset_test)

In [84]:
length

15993

In [85]:
y_pred = []
y_true = []
for i in range(length) : 
    pred = predict(dataset_test[i][0])[0]
    y_pred.append(pred)
    y_true.append(dataset_test[i][1])

## 결과 저장

In [123]:
result = pd.DataFrame()
result["y_pred"] = y_pred
result["y_true"] = y_true

In [124]:
result.head()

,y_pred,y_true
0,2,2
1,16,16
2,6,6
3,18,18
4,12,12


In [125]:
result.dtypes

y_pred     int64
y_true    object
dtype: object

In [127]:
#collapse-hide
result["y_true"]=result["y_true"].astype(int)

In [128]:
result.dtypes

y_pred    int64
y_true    int64
dtype: object

* 모델 입력시 토픽 넘버에 1을 빼줬으므로 다시 더해줌  

In [129]:
result["y_true"] = result["y_true"]+1
result["y_pred"] = result["y_pred"]+1

In [130]:
#collapse-hide
result.head()

,y_pred,y_true
0,3,3
1,17,17
2,7,7
3,19,19
4,13,13


* 1을 더한 토픽값이 잘 생성되었는지 결과 확인 

In [132]:
result["y_true"].unique()

array([ 3, 17,  7, 19, 13,  8, 22, 12,  4,  9, 11, 23,  1,  6, 20, 16,  5,
       21,  2, 10, 18, 14, 15])

* 결과 파일 저장 (14시간 걸린 파일이나 목숨보다 소중히 하자...)

In [134]:
result.to_csv("predict 결과.csv")

## Confusion matrix report 작성 

In [138]:
from sklearn.metrics import classification_report
import pandas as pd

In [190]:
result = pd.read_csv("predict 결과.csv")

In [145]:
y_true = result["y_true"].to_list()
y_pred = result["y_pred"].to_list()

In [167]:
topic_number = list(range(1,24))
topic_number = list(map(str,topic_number))
print(topic_number)

['1', '2', '3', '4', '5', '6', '7', '8', '9', '10', '11', '12', '13', '14', '15', '16', '17', '18', '19', '20', '21', '22', '23']


In [175]:
target_names = []
for i in range((len(topic_number))) :
      target_names.append("토픽 " + topic_number[i])

In [181]:
report = classification_report(y_true, y_pred, target_names=target_names,output_dict=True)

In [186]:
report_df = pd.DataFrame(report).transpose()

In [189]:
#collapse-hide
report_df

,precision,recall,f1-score,support
토픽 1,0.955319,0.958378,0.956846,937.000000
토픽 2,0.938395,0.933048,0.935714,702.000000
토픽 3,0.950920,0.943683,0.947288,657.000000
토픽 4,0.944134,0.926535,0.935252,912.000000
토픽 5,0.919877,0.914242,0.917051,653.000000
토픽 6,0.947257,0.949260,0.948258,946.000000
토픽 7,0.932357,0.946214,0.939234,1413.000000
토픽 8,0.951456,0.974586,0.962882,905.000000
토픽 9,0.960767,0.968366,0.964551,1138.000000
토픽 10,0.965574,0.937898,0.951535,628.000000


In [188]:
report_df.to_csv("분석결과report.csv")

* 분석이 끝났다. 맨 처음 데이터를 증강시키 않은 상태에서 5, 10 배 씩 증가시켜 모델을 적합하였다. 

* 총 3 가지 경우에 대해 모델링을 한 시간을 종합해보면 하.... 총 30 시간은 넘은 것 같다. 중간에 colab 알람 못 듣고 자서 모델 다시 돌릴 땐 정말 창문을 부수고 싶었다. 결과가 나와서 다행이다. 참고한 블로그와 코드 링크는 아래에 첨부하겠습니당

 # 참고

[1] [KoBert 모델링](https://hoit1302.tistory.com/159#3.%EB%8D%B0%EC%9D%B4%ED%84%B0%EC%85%8B%EC%84%A4%EB%AA%85,%EC%BD%94%EB%94%A9%ED%99%98%EA%B2%BD)

[2] [데이터 증강](https://github.com/catSirup/KorEDA/tree/master)